In [1]:
import pandas as pd
import numpy as np
import requests
import json
from config import api_key
from pprint import pprint
from config import gkey

In [3]:
cities_df = pd.read_csv("./chipotle_cities.csv")
cities_df.head()

,CITY,STATE
0,AUBURN,ALABAMA
1,BIRMINGHAM,ALABAMA
2,CULLMAN,ALABAMA
3,HOOVER,ALABAMA
4,HUNTSVILLE,ALABAMA


In [10]:
## Chipotle ##
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

def searchChipotle(city:str, state:str) -> requests.Response:

# geocoordinates

    target_search = f"Chipotle in {city}, {state}"

# set up a parameters dictionary
    params = {
        "query":target_search,
        "key": gkey    
    }

    response = requests.get(base_url, params=params)

    return response

In [11]:
responses = [searchChipotle(row.CITY, row.STATE) for _, row in cities_df.iterrows()]

In [12]:
test_response = responses[-1].json()['results'][0]

In [13]:
test_response

{'business_status': 'OPERATIONAL',
 'formatted_address': '1508 Dell Range Blvd, Cheyenne, WY 82009, United States',
 'geometry': {'location': {'lat': 41.160718, 'lng': -104.805699},
  'viewport': {'northeast': {'lat': 41.16206502989272,
    'lng': -104.8044708201073},
   'southwest': {'lat': 41.15936537010728, 'lng': -104.8071704798927}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
 'id': '77139f9a831aea3e92e3074c2583a6f9b51881ae',
 'name': 'Chipotle Mexican Grill',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 2788,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111705021100683327569">Robert Whittaker</a>'],
   'photo_reference': 'CmRaAAAAu_t_vRi4wSioLiXy2QPZWvjNszoMXmAN4yy_jAN6UdN5CbtHaKD-b-_V76YOpvUhGyaWcri4JnxVaVCFJszHqnExLmafogjXZM_2a-iFjLIu5qN1sL2Lm5-43JJLq_taEhD8y-HX3JC2MrHaGT7MCSUKGhQFwQz_GE26wpvDZOaIrKuTTwF9OQ',
   'width': 4032}],
 'place_id': 'ChIJm0KC7FY6b4cR3UWVlODrl9I',
 'plus_code': {'compound_c

In [14]:
def formatresponse(test_response): 
    last_results = {
    'Address':test_response["formatted_address"],'Latitude':test_response['geometry']['location']['lat'], 'Longitude':test_response['geometry']['location']['lng'],'Rating':test_response['rating'],'User Rating Count':test_response['user_ratings_total']}
    
    return last_results

In [15]:
flat_responses = []

for response in responses:
    flat_responses.extend(response.json()['results'])

In [16]:
neat_list = [formatresponse(response) for response in flat_responses]

In [17]:
neat_df = pd.DataFrame(neat_list)

In [18]:
neat_df.shape

(5162, 5)

In [19]:
unique_restaurants = neat_df.drop_duplicates()

unique_restaurants.head(20)

,Address,Latitude,Longitude,Rating,User Rating Count
0,"346 W Magnolia Ave, Auburn, AL 36832, United S...",32.606813,-85.487328,4.2,410
1,"2125 Interstate Dr, Opelika, AL 36801, United ...",32.616809,-85.404479,4.2,305
2,"300 20th St S, Birmingham, AL 35233, United St...",33.509721,-86.802756,4.1,748
3,"4719 US-280, Birmingham, AL 35242, United States",33.422582,-86.698279,4.1,424
4,"3220 Morrow Rd, Birmingham, AL 35235, United S...",33.595581,-86.647437,4.2,622
5,"1759 Montgomery Hwy, Hoover, AL 35244, United ...",33.378958,-86.803802,4.3,752
6,"1031 Montgomery Hwy #111, Vestavia Hills, AL 3...",33.439069,-86.788285,4.2,234
7,"1821 Cherokee Ave SW, Cullman, AL 35055, Unite...",34.154134,-86.841220,3.7,63
12,"5900 University Dr NW D2, Huntsville, AL 35806...",34.742319,-86.665720,4.1,1058
13,"3871 Airport Blvd, Mobile, AL 36608, United St...",30.675338,-88.143754,4.2,730


In [20]:
yelp_df = unique_restaurants.copy()

yelp_df.head(20)

,Address,Latitude,Longitude,Rating,User Rating Count
0,"346 W Magnolia Ave, Auburn, AL 36832, United S...",32.606813,-85.487328,4.2,410
1,"2125 Interstate Dr, Opelika, AL 36801, United ...",32.616809,-85.404479,4.2,305
2,"300 20th St S, Birmingham, AL 35233, United St...",33.509721,-86.802756,4.1,748
3,"4719 US-280, Birmingham, AL 35242, United States",33.422582,-86.698279,4.1,424
4,"3220 Morrow Rd, Birmingham, AL 35235, United S...",33.595581,-86.647437,4.2,622
5,"1759 Montgomery Hwy, Hoover, AL 35244, United ...",33.378958,-86.803802,4.3,752
6,"1031 Montgomery Hwy #111, Vestavia Hills, AL 3...",33.439069,-86.788285,4.2,234
7,"1821 Cherokee Ave SW, Cullman, AL 35055, Unite...",34.154134,-86.841220,3.7,63
12,"5900 University Dr NW D2, Huntsville, AL 35806...",34.742319,-86.665720,4.1,1058
13,"3871 Airport Blvd, Mobile, AL 36608, United St...",30.675338,-88.143754,4.2,730


In [22]:
yelp_df.to_csv("./chipotle_data.csv", header=True)